# 연구과제 예시 - 네이버 뉴스 수집

## #01. 준비과정

### [1] 패키지 참조

In [29]:
import datetime as dt
import requests
import os
import time
import re
from bs4 import BeautifulSoup
import markdownify as mf
from tqdm.notebook import tqdm
import concurrent.futures as futures

### [2] 필요한 변수 정의

#### (1) 카테고리별 URL 패턴

네이버 뉴스(`https://news.naver.com`)에서 카테고리별로 기사 목록 페이지의 URL을 확인하여 패턴을 찾음

| 분류 | 기사목록 URL |
|---|---|
| 정치 | https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100 |
| 경제 | https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101 |
| 사회 | https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102 |
| 생활, 문화 | https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103 |

> `sid1`으로 전달되는 숫자값에 의해 카테고리가 분류됨을 확인

#### (2) 페이지별 URL 패턴

```
https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103#&date=%2000:00:00&page=1
https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103#&date=%2000:00:00&page=3
https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103#&date=%2000:00:00&page=5
```

> 카테고리를 의미하는 `sid1`값이 고정된 상태에서 `page`변수의 값이 1단위로 증가함을 확인

In [2]:
# 네이버 뉴스의 카테고리별 sid값
SID1 = {"IT,과학" : 105, "경제" : 101, "사회" : 102, "생활,문화" : 103, "세계" : 104, "정치" : 100}

# 수집하고자하는 페이지 범위
PAGES = range(1, 11)

# 네이버 뉴스 기사 목록 URL
URL_FORMAT = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid1}#&date=%2000:00:00&page={page}'

### [3] 세션 생성

In [3]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

## #02. 전체 기사 목록 가져오기

### [1] 함수 정의

In [11]:
def getArticleList(session, category, url):
    try:
        r = session.get(url)
        
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)
        
    r.encoding = "euc-kr"
    soup = BeautifulSoup(r.text)
    #print(soup)
    
    
    # 뉴스 기사 제목을 위한 CSS 선택자
    selector = ".sh_text > .sh_text_headline,  .cluster_text > .cluster_text_headline,  .type06_headline dt:not(.photo),  .list_text_inner .list_tit,  .list_txt a"
    
    # 뉴스기사 제목 추출
    headlines = soup.select(selector)
    #print(headlines)
    
    
    # 각 페이지별로 추출된 뉴스기사의 본문 URL
    urls = []
    for h in headlines:
        if "href" in h.attrs:
            item = {'category': category, 'title': h.text.strip(), 'url': h.attrs['href']}
            
            # 수집된 URL이 정정기사 리스트 페이지로의 링크인 경우는 제외
            # ex) https://news.naver.com/main/ombudsman/revisionArticleList.naver?mid=omb
            if item['url'][-7:] == "mid=omb":
                print("제외 URL:", item['url'])
                continue
            
            urls.append(item)
        
    time.sleep(0.05)
        
    return urls

#### 함수 테스트

In [12]:
getArticleList(session, "사회", "https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102")

제외 URL: https://news.naver.com/main/ombudsman/errorArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/revisionArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/unfairArticleList.naver?mid=omb


[{'category': '사회',
  'title': '내일 전국 비바람…수도권 이틀간 강수량 최대 80㎜',
  'url': 'https://n.news.naver.com/mnews/article/277/0005354515?sid=102'},
 {'category': '사회',
  'title': '완도 해조류·전복산업특구 ‘탁월 특구’ 선정',
  'url': 'https://n.news.naver.com/mnews/article/003/0012266241?sid=102'},
 {'category': '사회',
  'title': '“군인은 3000원 더 내세요”… 온라인서 뭇매 맞은 고깃집',
  'url': 'https://n.news.naver.com/mnews/article/023/0003804671?sid=102'},
 {'category': '사회',
  'title': '초등 학부모 채팅방 ‘아이들 살해’ 협박 글 올린 고교생.. 경찰, 구속영장',
  'url': 'https://n.news.naver.com/mnews/article/023/0003804599?sid=102'},
 {'category': '사회',
  'title': '익산 산란계 농장 두 곳 고병원성 AI…누적 10건',
  'url': 'https://n.news.naver.com/mnews/article/422/0000634302?sid=102'},
 {'category': '사회',
  'title': '한 총리 "청년주택 청약 당첨 시 2%대 금리로 40년까지 대출"',
  'url': 'https://n.news.naver.com/mnews/article/421/0007233037?sid=102'},
 {'category': '사회',
  'title': '"죄질이 극히 불량"…40대 여성 납치·성폭행한 중학생 징역형',
  'url': 'https://n.news.naver.com/mnews/article/088/0000851416?sid=102'},
 {'ca

### [2] 기사 URL 추출

In [13]:
articles = []

# 카테고리수 x 페이지 수가 전체 가져와야 할 수량
progress = tqdm(total=len(SID1) * len(PAGES))

for c in list(SID1.keys()):
    for p in PAGES:
        url = URL_FORMAT.format(sid1=SID1[c], page=p)
        #print(url)
        articles += getArticleList(session, c, url)
        progress.update()
        
print("수집된 URL수:", len(articles)) 
articles

  0%|          | 0/60 [00:00<?, ?it/s]

제외 URL: https://news.naver.com/main/ombudsman/errorArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/revisionArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/unfairArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/errorArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/revisionArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/unfairArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/errorArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/revisionArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/unfairArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/errorArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/revisionArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/unfairArticleList.naver?mid=omb
제외 URL: https://news.naver.com/main/ombudsman/errorArticleLi

[{'category': 'IT,과학',
  'title': "개인정보위, '해킹 무방비' 안전조치 의무 위반 업체 2곳 제재",
  'url': 'https://n.news.naver.com/mnews/article/001/0014389726?sid=105'},
 {'category': 'IT,과학',
  'title': '초전도학회 검증위 “‘LK-99’ 상온 상압 초전도체 근거 없어”',
  'url': 'https://n.news.naver.com/mnews/article/056/0011621374?sid=105'},
 {'category': 'IT,과학',
  'title': '네이처, 2023 과학계 화제의 인물에…‘챗GPT’ 선정',
  'url': 'https://n.news.naver.com/mnews/article/016/0002238643?sid=105'},
 {'category': 'IT,과학',
  'title': '카카오, 신임 대표로 정신아 카카오벤처스 대표 내정',
  'url': 'https://n.news.naver.com/mnews/article/366/0000954583?sid=105'},
 {'category': 'IT,과학',
  'title': 'KT의 갤럭시S23 FE 고객 절반 이상, 구독서비스 이용',
  'url': 'https://n.news.naver.com/mnews/article/008/0004973978?sid=105'},
 {'category': 'IT,과학',
  'title': "'美상장 추진' 네이버웹툰…다시 조명받는 이해진 역할",
  'url': 'https://n.news.naver.com/mnews/article/018/0005637559?sid=105'},
 {'category': 'IT,과학',
  'title': "구글 최신 AI 언어모델 '제미나이' 기업용 서비스에 탑재",
  'url': 'https://n.news.naver.com/mnews/article/001/00143892

In [ ]:
q = len(articles)

for i in range(q-1, -1, -1):
    url = articles[i]['url']

    for j in range(i-1, -1, -1):
        if url == articles[j]['url']:
            del(articles[i])
            break

r = len(articles)

print("before:", q, "after:", r)

- 리스트의 중복제거를 내장함수로 앞 순서대로 하게 되면 삭제된 원소에 의해 순서가 꼬일 우려가 있기 때문에 뒤에서부터 하는것이 원칙

## #03. 기사 본문 가져오기

### [1] 함수 정의

In [14]:
def getArticleBody(session, url):
    try:
        r = session.get(url)
        
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)
        
    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text)
    
    content = soup.select("#dic_area")
    
    if not content:
        print("컨텐츠 수집 실패 >>", url)
        return
    
    content = content[0]
    
    # 본문 요소에 포함되어 있는 불필요 항목 제거
    for target in content.find_all("table", {"class": "nbd_table"}):
        target.extract()
        
    for target in content.find_all("span", {"class": "end_photo_org"}):
        target.extract()
                    
    for target in content.find_all("strong", {"class": "media_end_summary"}):
        target.extract()
        
    for target in content.find_all("div", {"class": "highlightBlock"}):
        target.extract()
        
    for target in content.find_all("div", {"style": "display:none;"}):
        target.extract()
        
    for target in content.find_all("br"):
        target.replace_with("\n")
    
    #print(content.text.strip())
    
    #print(mf.markdownify(str(content)).strip())
    
    return content.text.strip()

#### 함수 테스트

In [15]:
txt = getArticleBody(session, "https://n.news.naver.com/mnews/article/018/0005637593?sid=102")
print(txt)

[이데일리 홍수현 기자] 중앙분리대 틈 사이를 기어 나와 무단횡단을 시도하는 보행자 모습이 포착돼 공분을 사고 있다.

14일 MBC는 대구 북구 침산동의 한 도로에서 지난 주말 찍힌 차량 블랙박스 영상을 보도하며 무단횡단에 대한 경각심을 일깨웠다. 

영상은 1차선으로 주행 중인 블랙박스 차량 시점으로 시작된다. 이때 갑자기 중앙분리대 밑에서 무언가 꿈틀하더니 쑥 올라와 운전자를 당황하게 만들었다. 중앙분리대 밑에서 기어나온 건 다름아닌 사람이었다. 

이 사람은 차량이 연이어 오는데도 아랑곳 않고 기어이 중앙분리대 밑에서 나와 무단횡단을 이어갔다. 사방이 어둑한 상태라 자칫 큰 사고로 이어질 수 있는 상황이었다. 

이런 일이 이번이 처음은 아니다. 지난달 인천에서도 중앙분리대 밑을 기어나오는 어르신이 목격됐다.

당시에도 이 모습을 본 차량이 급히 속도를 줄이면서 다행히 사고로 이어지진 않았지만 노인을 향한 비판의 목소리가 거셌다.


### [2] 기사 본문 수집

하나의 기사를 수집하는 즉시 파일로 저장한다.

약 10만건의 기사 본문을 수집하여 빈 리스트에 저장할 경우 컴퓨터 메모리 낭비가 심하기 때문에 1건씩 파일로 기록하고 별도의 메모리 공간에는 저장하지 않도록 하는 것이 이상적인 처리이다.

#### 파일이 저장될 폴더 생성

In [17]:
# 파일이 저장될 폴더 생성
dirname = dt.datetime.now().strftime("뉴스기사_수집_%y%m%d_%H%M%S")

if not os.path.exists(dirname):
    os.mkdir(dirname)

#### 뉴스기사 수집

파일이름에 포함 될 수 없는 특수문자는 `\`, `/`, `:`, `*`, `?`, `<`, `>`, `|`, `"`, `'`, 이다. 

파이썬은 정규표현식을 지원하는 re 패키지를 제공한다. 

원본 문자열에서 정규표현식을 충족하는 글자를 다른 내용으로 치환하기

```python
import re
foo = re.sub(정규표현식, 변경할내용, 원본문자열)
```

```python
title = '<제목> "입"\'\'니다**'
filename = re.sub("[\\/:*?<>|\"\']", "", title) + '.xls'
```

In [37]:
progress = tqdm(total=len(articles))

with futures.ThreadPoolExecutor(max_workers=10) as executor:
    for i, v in enumerate(articles):
        #print(v)
        
        #print(v['category'])
        #print(v['title'])
        #print(v['url'])
        
        fu = executor.submit(getArticleBody, session, v['url'])
        result = fu.result()
        fu.add_done_callback(lambda x : progress.update())
        
        if result:
            title = re.sub("[\\/:*?<>|\"\']", "", v['title'])
            
            fname = "%s/[%s] %s.txt" % (dirname, v['category'], title)
            with open(fname, 'w', encoding='utf-8') as f:
                f.write(result)
                
            print(fname,"(이)가 생성되었습니다.")

  0%|          | 0/6829 [00:00<?, ?it/s]

뉴스기사_수집_231214_110556/[IT,과학] 개인정보위, 해킹 무방비 안전조치 의무 위반 업체 2곳 제재.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] 초전도학회 검증위 “‘LK-99’ 상온 상압 초전도체 근거 없어”.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] 네이처, 2023 과학계 화제의 인물에…‘챗GPT’ 선정.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] 카카오, 신임 대표로 정신아 카카오벤처스 대표 내정.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] KT의 갤럭시S23 FE 고객 절반 이상, 구독서비스 이용.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] 美상장 추진 네이버웹툰…다시 조명받는 이해진 역할.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] 구글 최신 AI 언어모델 제미나이 기업용 서비스에 탑재.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] 아이폰 통녹 이어 실시간 통역콜까지…SKT, 에이닷 업데이트.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] ASML 보유 네덜란드-韓 삼성·SK 반도체 동맹…TSMC 추월 희망가 부른다.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] AI 규제법 발효 앞둔 EU…HBM 수요에 미칠 영향은.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] D램 서버 삼성전자 역전한 SK하이닉스…DDR5 우위 주효.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과학] 佛 엔지니어의 유창한 한국어…’ST 터치GFX’ 알렸다.txt (이)가 생성되었습니다.
뉴스기사_수집_231214_110556/[IT,과

KeyboardInterrupt: 